In [1]:
import pandas as pd
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import search, print_breakdown


df_nhtsa = await search(
  min_model_year=2017,
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36764 VINs (filter_comment=None, include_openpilot=True, skipped=163, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/36762 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              996
Bronco Sport       1500
C-Max                 5
Continental          23
Corsair             986
Ecosport            406
Edge               2685
Escape             3568
Expedition          835
Expedition MAX      733
Explorer           3364
F-150              8638
F-250              1720
F-350              1158
F-450               207
F-550                 1
Fiesta              195
Flex                189
Focus               169
Fusion             1163
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             952
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus              122
Transit            1391
Transit Connect     414
dtype: int64


In [2]:
from notebooks.ford.ecu import FordEcu
from panda.python.uds import DATA_IDENTIFIER_TYPE


def get_platform_code(row):
  abd = AsBuiltData.from_vin(row['VIN'])

  eps_fw = abd.get_identifier(FordEcu.PowerSteeringControlModule, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if eps_fw is None:
    platform_code = None
  else:
    prefix, core, _ = eps_fw.split('-')
    platform_code = f'{core}-{prefix}'

  return pd.Series({
    'PlatformCode': platform_code,
  })


df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df = df.join(df_nhtsa.apply(get_platform_code, axis=1))
df.head()

,VIN,CarName,PlatformCode
0,2FMPK4J99PBA24766,FORD Edge 2023,14D003-M2GC
1,1FTEW1CP6LKD67075,FORD F-150 2020,14D003-KL3V
2,1FA6P8CF1R5414396,FORD Mustang 2024,14D003-PR3C
3,3FMTK4SE2PMA08028,FORD Mustang Mach-E 2023,14D003-LJ9C
4,1FMSK8DH5NGA37606,FORD Explorer 2022,14D003-M1MC


In [3]:
df_groups = df.groupby(['CarName', 'PlatformCode'], dropna=False).size().reset_index(name='count')
df_groups.head()

,CarName,PlatformCode,count
0,FORD Bronco 2021,14D003-MB3C,99
1,FORD Bronco 2022,14D003-MB3C,87
2,FORD Bronco 2022,14D003-NB3C,118
3,FORD Bronco 2022,14D003-NL38,6
4,FORD Bronco 2023,14D003-NB3C,605


In [4]:
for group in df_groups.groupby('PlatformCode', dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

14C217-AE81
CarName: ['FORD Fiesta 2017' 'FORD Fiesta 2018' 'FORD Fiesta 2019']

14C217-CV6T
CarName: ['FORD Focus 2017' 'FORD Focus 2018' 'FORD Transit Connect 2018']

14C217-GN15
CarName: ['FORD Ecosport 2018' 'FORD Ecosport 2019' 'FORD Ecosport 2020'
 'FORD Ecosport 2021' 'FORD Ecosport 2022']

14C217-HV6T
CarName: ['FORD C-Max 2018' 'FORD Escape 2018' 'FORD Escape 2019' 'FORD Focus 2018'
 'FORD Transit Connect 2018']

14D003-GG13
CarName: ['FORD Flex 2018' 'FORD Flex 2019' 'FORD Taurus 2018' 'FORD Taurus 2019'
 'LINCOLN MKT 2019']

14D003-GJ7C
CarName: ['LINCOLN MKC 2019']

14D003-HB53
CarName: ['FORD Explorer 2017' 'FORD Explorer 2018' 'FORD Explorer 2019']

14D003-HG9C
CarName: ['FORD Edge 2018' 'FORD Fusion 2018']

14D003-HR3C
CarName: ['FORD F-150 2017']

14D003-JL14
CarName: ['FORD Expedition 2018' 'FORD Expedition MAX 2018' 'FORD Mustang 2018']

14D003-JL34
CarName: ['FORD F-150 2018']

14D003-JR3C
CarName: ['FORD F-150 2018' 'FORD Mustang 2018' 'FORD Mustang 2019'
 'FORD Ran

In [12]:
from notebooks.ford.platforms import find_openpilot_platform


def get_openpilot_platform(row):
  return pd.Series({
    'CarInfoPlatform': find_openpilot_platform(row['CarName']),
  })


df_openpilot_platforms = df.join(df.apply(get_openpilot_platform, axis=1))
df_openpilot_platforms.head()

,VIN,CarName,PlatformCode,CarInfoPlatform
0,2FMPK4J99PBA24766,FORD Edge 2023,14D003-M2GC,None
1,1FTEW1CP6LKD67075,FORD F-150 2020,14D003-KL3V,None
2,1FA6P8CF1R5414396,FORD Mustang 2024,14D003-PR3C,None
3,3FMTK4SE2PMA08028,FORD Mustang Mach-E 2023,14D003-LJ9C,FORD MUSTANG MACH-E 1ST GEN
4,1FMSK8DH5NGA37606,FORD Explorer 2022,14D003-M1MC,FORD EXPLORER 6TH GEN


In [14]:
df_openpilot_platforms['CarInfoPlatform'].value_counts(dropna=False)

CarInfoPlatform
None                           27179
FORD EXPLORER 6TH GEN           3332
FORD ESCAPE 4TH GEN             2164
FORD F-150 14TH GEN             1556
FORD BRONCO SPORT 1ST GEN        930
FORD MUSTANG MACH-E 1ST GEN      813
FORD MAVERICK 1ST GEN            788
Name: count, dtype: int64

In [15]:
for group in df_openpilot_platforms.groupby('CarInfoPlatform', dropna=False):
  print(group[0])
  print(f'CarName: {group[1]["CarName"].unique()}')
  print()

FORD BRONCO SPORT 1ST GEN
CarName: ['FORD Bronco Sport 2021' 'FORD Bronco Sport 2022']

FORD ESCAPE 4TH GEN
CarName: ['FORD Escape 2020' 'FORD Escape 2022' 'FORD Escape 2021']

FORD EXPLORER 6TH GEN
CarName: ['FORD Explorer 2022' 'FORD Explorer 2020' 'FORD Explorer 2023'
 'LINCOLN Aviator 2021' 'FORD Explorer 2021' 'LINCOLN Aviator 2020']

FORD F-150 14TH GEN
CarName: ['FORD F-150 2023']

FORD MAVERICK 1ST GEN
CarName: ['FORD Maverick 2023' 'FORD Maverick 2022']

FORD MUSTANG MACH-E 1ST GEN
CarName: ['FORD Mustang Mach-E 2023' 'FORD Mustang Mach-E 2022'
 'FORD Mustang Mach-E 2021']

nan
CarName: ['FORD Edge 2023' 'FORD F-150 2020' 'FORD Mustang 2024' 'FORD Ranger 2020'
 'FORD Bronco Sport 2023' 'LINCOLN Corsair 2023' 'LINCOLN Corsair 2020'
 'FORD Fusion 2020' 'FORD Mustang 2020' 'FORD Escape 2023'
 'FORD F-150 2022' 'FORD F-250 2022' 'FORD Transit 2018' 'FORD F-150 2021'
 'FORD Ranger 2019' 'FORD Expedition 2020' 'FORD Edge 2024'
 'FORD Expedition 2019' 'FORD F-250 2019' 'FORD Focus 20

In [ ]:
PSCM_A = '14C217'
PSCM_B = '14D003'

def predict_platform(platform_code: str) -> str | None:
  if platform_code is None:
    return None

  core, prefix = platform_code.split('-')

  # Not recognised
  if core not in (PSCM_A, PSCM_B):
    return None

  # Not supported in openpilot
  if core == PSCM_A:
    return None

  raise ValueError(f'Unknown platform code: {platform_code}')